In [6]:
import requests
import pandas as pd
from urllib.parse import quote
from bs4 import BeautifulSoup

In [7]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
res = requests.get(url)
soup = BeautifulSoup(res.text)

In [8]:
lis = soup.select('.subject')
len(lis)

101

In [9]:
from selenium import webdriver

driver = webdriver.Chrome('C:/Users/YONSAI/Desktop/chromedriver_win32/chromedriver.exe')
driver.get(url)

C:\Users\YONSAI\AppData\Local\Temp\ipykernel_11992\1302889787.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/YONSAI/Desktop/chromedriver_win32/chromedriver.exe')


In [10]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

body = driver.find_element(By.TAG_NAME,'body')
for _ in range(3):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [11]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [12]:
len(driver.page_source), len(res.text)

(306820, 288946)

- Step 3

In [13]:
# rank
tr = trs[0]
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [14]:
# category
category = tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [15]:
# channel명
channel = tr.select_one('.subject > h1 > a').get_text().strip()
channel

'BLACKPINK'

In [16]:
# 구독자
subscriber_cnt = tr.select_one('.subscriber_cnt').get_text().strip()
subscriber_cnt

'8730만'

In [17]:
# View, Video
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
view, video

('305억3350만', '507개')

In [20]:
# 숫자로 바꿔주는 함수
# def convert(s):

SyntaxError: incomplete input (59084177.py, line 2)

In [21]:
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [22]:
convert('355억8260만')

35582600000


step. 4

In [23]:
lines = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber_cnt = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    lines.append({
        '순위':rank, '카테고리':category, '채널명':channel, '구독자':subscriber_cnt,
        '시청자수':view, '영상갯수':video
    })
    

In [24]:
df1 = pd.DataFrame(lines)
df1.tail(7)

,순위,카테고리,채널명,구독자,시청자수,영상갯수
93,94,음식/요리/레시피,떵개떵,4890000,2981590000,4910
94,96,키즈/어린이,ToyMart TV,4860000,1993230000,687
95,95,키즈/어린이,뽀로로(Pororo),4860000,6853150000,4160
96,97,애완/반려동물,SBS TV동물농장x애니멀봐,4830000,4638100000,4164
97,98,키즈/어린이,CoCosToy 꼬꼬스토이,4770000,2983640000,686
98,99,음식/요리/레시피,Cooking tree 쿠킹트리,4760000,487580000,1450
99,100,미분류,매일맛나 delicious day,4740000,482360000,325


- 두번째 페이지

In [25]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(2)
driver.get(url)
time.sleep(2)
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [ ]:
driver.close()

In [27]:

lines = []
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
for page in range(1, 11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('.aos-init')

    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        lines.append({
            '순위':rank, '카테고리':category, '채널명':channel,
            '구독자수':subscriber, '조회수':view, '비디오':video
        })

driver.close()

C:\Users\YONSAI\AppData\Local\Temp\ipykernel_11992\3158818478.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')


In [28]:
df = pd.DataFrame(lines)
df.shape

(1000, 6)

In [29]:
df.tail()

,순위,카테고리,채널명,구독자수,조회수,비디오
995,991,회사/오피셜,LG Uplus,730000,1379980000,2473
996,998,BJ/인물/연예인,케이,730000,572670000,3304
997,1001,미분류,백크 Baekk,720000,457320000,1106
998,999,음악/댄스/가수,Crush,720000,395460000,131
999,997,BJ/인물/연예인,bobaepapa - 보배아빠,720000,348440000,393


- step 6

In [30]:
df.to_csv('data/20230425_YouTube랭킹.csv', index=False)